# Add Embeddings to an Existing Table


In this example we will extend an existing table with embeddings computed from a pre-trained model.

- Write an initial table containing a single column of image URLs.
- Write a new table containing the input URLs and the embeddings computed from a pre-trained model.
- Apply dimensionality reduction to the extended table to get a final table containing the URLs, the embeddings, and the reduced embeddings.

In [ ]:
from pathlib import Path
import tlc
import tqdm

from tools.common import examples_root

## Write the initial table

We write a simple table containing a single column of image URLs from our COCO-128 dataset.

In [ ]:
data_path = (examples_root() / "data/coco128/images").absolute().as_posix()
project_name = "add-embeddings"
dataset_name = "coco128"

table = tlc.Table.from_image_folder(
    data_path,
    include_label_column=False,
    table_name="initial",
    dataset_name=dataset_name,
    project_name=project_name,
    add_weight_column=False,
    description="COCO128 dataset",
)


## Extend the table with embeddings from a pre-trained model

We will use the ViT model pre-trained on ImageNet to compute embeddings for the images in the table.
A benefit of using this model is that meaningful embeddings can be extracted easily using the `last_hidden_state` attribute of the model output.

In [ ]:
from tools.embeddings import add_embeddings_to_table

table.map(lambda x: x.convert("RGB"))
extended_table = add_embeddings_to_table(table, batch_size=32)

## Reduce the embeddings to 2 dimensions

Finally we reduce the embedding-column to 2 dimensions using UMAP. The result is a table containing the URLs, the embeddings, and the reduced embeddings.

In [ ]:
reduced_table = tlc.reduce_embeddings(
    extended_table,
    method="pacmap",
    n_components=2,
    retain_source_embedding_column=False,
)

print(reduced_table.table_rows[0].keys()) # The row-view of the reduced table contains both the embeddings and the reduced embeddings

In [ ]:
reduced_table.model_url